In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("spam.csv")
df.shape

(5572, 2)

In [3]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df["Spam"] = df.Category.apply(lambda x: 1 if x == "spam" else 0)
df.head()

,Category,Message,Spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.Message, df.Spam, test_size=0.2)

X_test.shape, X_train.shape

((1115,), (4457,))

In [16]:
print("Datatype: ", type(X_train))
print("Datatype of X_train.values: ", type(X_train.values))
print("\n\nX_train[:4]: \n", X_train[:4])
print("\n\nX_train.values[:4]: \n", X_train.values[:4])

Datatype:  <class 'pandas.core.series.Series'>
Datatype of X_train.values:  <class 'numpy.ndarray'>


X_train[:4]: 
 125                Ü predict wat time ü'll finish buying?
3324    She said,'' do u mind if I go into the bedroom...
3291    My tuition is at 330. Hm we go for the 1120 to...
1705    What can i do? Might accidant tookplace betwee...
Name: Message, dtype: object


X_train.values[:4]: 
 ["Ü predict wat time ü'll finish buying?"
 "She said,'' do u mind if I go into the bedroom for a minute ? '' ''OK'', I sed in a sexy mood. She came out 5 minuts latr wid a cake...n My Wife,"
 'My tuition is at 330. Hm we go for the 1120 to 1205 one? Do you mind?'
 "What can i do? Might accidant tookplace between somewhere ghodbandar rd. Traffic moves slovely. So plz slip &amp; don't worry."]


Bag Of Words (CountVectorizer)

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer()
X_train_cv = v.fit_transform(X_train)
X_test_cv = v.transform(X_test)

To train a model we need array of values lets check the type of X_test_cv

In [18]:
type(X_test_cv )

scipy.sparse._csr.csr_matrix

In [31]:
X_train_cv.shape
# X_test_cv.toarray()[:2][0]


(4457, 7732)

In [26]:
X_test_np = X_test_cv.toarray()
X_train_np = X_train_cv.toarray()
X_train_np.shape

(4457, 7732)

In [38]:
np.where(X_train_np[0] != 0)

(array([1575, 2844, 4155, 5365, 6887, 7384], dtype=int64),)

In [37]:
X_train.values[0]

"Ü predict wat time ü'll finish buying?"

In [48]:
v.get_feature_names_out()[1575]

'buying'

In [52]:
for i in (np.where(X_train_np[2]!=0)):
    print(v.get_feature_names_out()[i])

['1120' '1205' '330' 'at' 'do' 'for' 'go' 'hm' 'is' 'mind' 'my' 'one'
 'the' 'to' 'tuition' 'we' 'you']


In [54]:
X_train.values[2]

'My tuition is at 330. Hm we go for the 1120 to 1205 one? Do you mind?'

Train a ML Model (NAIVE BAYES)

In [56]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB
mnb = MultinomialNB()
gnb = GaussianNB()
mnb.fit(X_train_np, y_train)
gnb.fit(X_train_np, y_train)

GaussianNB()

Performance of GaussianNB

In [59]:
from sklearn.metrics import classification_report
y_pred_g = gnb.predict(X_test_np)
print(classification_report(y_test,y_pred_g))

              precision    recall  f1-score   support

           0       0.98      0.90      0.94       950
           1       0.61      0.92      0.73       165

    accuracy                           0.90      1115
   macro avg       0.80      0.91      0.83      1115
weighted avg       0.93      0.90      0.91      1115



Performance of MultinomialNB

In [60]:
y_pred_m = mnb.predict(X_test_np)
print(classification_report(y_test,y_pred_m))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       950
           1       0.96      0.93      0.94       165

    accuracy                           0.98      1115
   macro avg       0.97      0.96      0.97      1115
weighted avg       0.98      0.98      0.98      1115



In [65]:
emails =    ["Congratulations! You've won a free iPhone. Click here to claim your prize.",
            "URGENT: Your bank account has been compromised. Please click the link to verify your information.",
            "Get rich quick! Make $10,000 in one week with our amazing investment opportunity.",
            "Hi, this is John from XYZ Company. We're offering a special discount on our products this week.",
            "Your Netflix account has been suspended. Please update your payment information to restore access."
            ]

In [66]:
mails = v.transform(emails)
mnb.predict(mails)

array([1, 1, 1, 0, 1], dtype=int64)

Train the model using sklearn pipeline and reduce number of lines of code

In [77]:
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ("vectorizer", CountVectorizer()),
    ("nb", MultinomialNB())
])

In [78]:
clf.fit(X_train.values,y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [79]:
y_pred = clf.predict(X_test)

In [80]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       950
           1       0.96      0.93      0.94       165

    accuracy                           0.98      1115
   macro avg       0.97      0.96      0.97      1115
weighted avg       0.98      0.98      0.98      1115

